In [0]:
service_credential = dbutils.secrets.get(scope="Secret02",key="ClientSecret")

spark.conf.set("fs.azure.account.auth.type.azureprojectmk2023.dfs.core.windows.net", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type.azureprojectmk2023.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set("fs.azure.account.oauth2.client.id.azureprojectmk2023.dfs.core.windows.net", "85c030e5-d1af-4660-961f-e52428c3133f")
spark.conf.set("fs.azure.account.oauth2.client.secret.azureprojectmk2023.dfs.core.windows.net", service_credential)
spark.conf.set("fs.azure.account.oauth2.client.endpoint.azureprojectmk2023.dfs.core.windows.net", "https://login.microsoftonline.com/d001e3df-c491-489f-9fe5-7e6bdf26c1aa/oauth2/token")

---------------------------------------------------------------------------
IllegalArgumentException                  Traceback (most recent call last)
File <command-1151741782894698>, line 1
----> 1 service_credential = dbutils.secrets.get(scope="Secret02",key="ClientSecret")
      3 spark.conf.set("fs.azure.account.auth.type.azureprojectmk2023.dfs.core.windows.net", "OAuth")
      4 spark.conf.set("fs.azure.account.oauth.provider.type.azureprojectmk2023.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")

File /databricks/python_shell/dbruntime/dbutils.py:240, in DBUtils.SecretsHandler.get(self, scope, key)
    239 def get(self, scope, key):
--> 240     return self.entry_point.getDbutils().preview().secret().get(scope, key)

File /databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py:1322, in JavaMember.__call__(self, *args)
   1316 command = proto.CALL_COMMAND_NAME +\
   1317     self.command_header +\
   1318     args_command +\


In [0]:
bronze_folder_path  = "abfss://bronze3@azureprojectmk2023.dfs.core.windows.net/"
silver_folder_path= "abfss://silver3@azureprojectmk2023.dfs.core.windows.net/"
gold_folder_path = "abfss://gold3@azureprojectmk2023.dfs.core.windows.net/"


In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import  col , regexp_replace


In [0]:
df = spark.read.format("delta").load(silver_folder_path+"data/address")

In [0]:
list1 = ["address" , "customer" , "customeraddress" , "product" , "productcategory" , "productdescription" , "productmodel" , "productmodelproductdescription" , "salesorderdetail" , "salesorderheader" ]
for i in range(len(list1)):
    df = spark.read.format("delta").load(silver_folder_path+"data/"+list1[i])
    column_names = df.columns
    for old_col_name in column_names:
        new_col_name = "".join(["_"  + char if char.isupper() and not old_col_name[i-1].isupper() else char for i, char in enumerate (old_col_name)]).lstrip("_")
        df = df.withColumnRenamed(old_col_name , new_col_name)
    df.write.format("delta").mode("overwrite").save(gold_folder_path+"data/"+list1[i]+"/")
    display(df)

Address_ID,Address_Line1,Address_Line2,City,State_Province,Country_Region,Postal_Code,rowguid,Modified_Date
9,8713 Yosemite Ct.,null,Bothell,Washington,United States,98011,{268AF621-76D7-4C78-9441-144FD139821A},2006-07-01
11,1318 Lasalle Street,null,Bothell,Washington,United States,98011,{981B3303-ACA2-49C7-9A96-FB670785B269},2007-04-01
25,9178 Jumping St.,null,Dallas,Texas,United States,75201,{C8DF3BD9-48F0-4654-A8DD-14A67A84D3C6},2006-09-01
28,9228 Via Del Sol,null,Phoenix,Arizona,United States,85004,{12AE5EE1-FC3E-468B-9B92-3B970B169774},2005-09-01
32,26910 Indela Road,null,Montreal,Quebec,Canada,H1Y 2H5,{84A95F62-3AE8-4E7E-BBD5-5A6F00CD982D},2006-08-01
185,2681 Eagle Peak,null,Bellevue,Washington,United States,98004,{7BCCF442-2268-46CC-8472-14C44C14E98C},2006-09-01
297,7943 Walnut Ave,null,Renton,Washington,United States,98055,{52410DA4-2778-4B1D-A599-95746625CE6D},2006-08-01
445,6388 Lake City Way,null,Burnaby,British Columbia,Canada,V5A 3A6,{53572F25-9133-4A8B-A065-102FF35416EE},2006-09-01
446,52560 Free Street,null,Toronto,Ontario,Canada,M4B 1V7,{801A1DFC-5125-486B-AA84-CCBD2EC57CA4},2005-08-01
447,22580 Free Street,null,Toronto,Ontario,Canada,M4B 1V7,{88CEE379-DBB8-433B-B84E-A35E09435500},2006-08-01


Customer_ID,Name_Style,Title,First_Name,Middle_Name,Last_Name,Suffix,Company_Name,Sales_Person,Email_Address,Phone,Password_Hash,Password_Salt,rowguid,Modified_Date
1,False,Mr.,Orlando,N.,Gee,null,A Bike Store,adventure-works\pamela0,orlando0@adventure-works.com,245-555-0173,L/Rlwxzp4w7RWmEgXX+/A7cXaePEPcp+KwQhl2fJL7w=,1KjXYs4=,{3F5AE95E-B87D-4AED-95B4-C3797AFCB74F},2005-08-01
2,False,Mr.,Keith,null,Harris,null,Progressive Sports,adventure-works\david8,keith0@adventure-works.com,170-555-0127,YPdtRdvqeAhj6wyxEsFdshBDNXxkCXn+CRgbvJItknw=,fs1ZGhY=,{E552F657-A9AF-4A7D-A645-C429D6E02491},2006-08-01
3,False,Ms.,Donna,F.,Carreras,null,Advanced Bike Components,adventure-works\jillian0,donna0@adventure-works.com,279-555-0130,LNoK27abGQo48gGue3EBV/UrlYSToV0/s87dCRV7uJk=,YTNH5Rw=,{130774B1-DB21-4EF3-98C8-C104BCD6ED6D},2005-09-01
4,False,Ms.,Janet,M.,Gates,null,Modular Cycle Systems,adventure-works\jillian0,janet1@adventure-works.com,710-555-0173,ElzTpSNbUW1Ut+L5cWlfR7MF6nBZia8WpmGaQPjLOJA=,nm7D5e4=,{FF862851-1DAA-4044-BE7C-3E85583C054D},2006-07-01
5,False,Mr.,Lucy,null,Harrington,null,Metropolitan Sports Supply,adventure-works\shu0,lucy0@adventure-works.com,828-555-0186,KJqV15wsX3PG8TS5GSddp6LFFVdd3CoRftZM/tP0+R4=,cNFKU4w=,{83905BDC-6F5E-4F71-B162-C98DA069F38A},2006-09-01
6,False,Ms.,Rosmarie,J.,Carroll,null,Aerobic Exercise Company,adventure-works\linda3,rosmarie0@adventure-works.com,244-555-0112,OKT0scizCdIzymHHOtyJKQiC/fCILSooSZ8dQ2Y34VM=,ihWf50M=,{1A92DF88-BFA2-467D-BD54-FCB9E647FDD7},2007-09-01
7,False,Mr.,Dominic,P.,Gash,null,Associated Bikes,adventure-works\shu0,dominic0@adventure-works.com,192-555-0173,ZccoP/jZGQm+Xpzc7RKwDhS11YFNybwcPVRYTSNcnSg=,sPoUBSQ=,{03E9273E-B193-448E-9823-FE0C44AEED78},2006-07-01
10,False,Ms.,Kathleen,M.,Garza,null,Rural Cycle Emporium,adventure-works\jos�1,kathleen0@adventure-works.com,150-555-0127,Qa3aMCxNbVLGrc0b99KsbQqiVgwYDfHcsK9GZSUxcTM=,Ls05W3g=,{CDB6698D-2FF1-4FBA-8F22-60AD1D11DABD},2006-09-01
11,False,Ms.,Katherine,null,Harding,null,Sharp Bikes,adventure-works\jos�1,katherine0@adventure-works.com,926-555-0159,uRlorVzDGNJIX9I+ehTlRK+liT4UKRgWhApJgUMC2d4=,jpHKbqE=,{750F3495-59C4-48A0-80E1-E37EC60E77D9},2005-08-01
12,False,Mr.,Johnny,A.,Caprio,Jr.,Bikes and Motorbikes,adventure-works\garrett1,johnny0@adventure-works.com,112-555-0191,jtF9jBoFYeJTaET7x+eJDkd7BzMz15Wo9odbGPBaIak=,wVLnvHo=,{947BCAF1-1F32-44F3-B9C3-0011F95FBE54},2006-08-01


Customer_ID,Address_ID,Address_Type,rowguid,Modified_Date
29485,1086,Main Office,{16765338-DBE4-4421-B5E9-3836B9278E63},2007-09-01
29486,621,Main Office,{22B3E910-14AF-4ED5-8B4D-23BBE757414D},2005-09-01
29489,1069,Main Office,{A095C88B-D7E6-4178-A078-2ECA44214801},2005-07-01
29490,887,Main Office,{F12E1702-D897-4035-B614-0FE2C72168A9},2006-09-01
29492,618,Main Office,{5B3B3EB2-3F43-47ED-A20C-23697DABF23B},2006-12-01
29494,537,Main Office,{492D92B6-31AF-47EA-8E00-7C11C7CCC20D},2005-09-01
29496,1072,Main Office,{0A66B0F3-24BC-4148-9661-26E935CEC99A},2007-09-01
29497,889,Main Office,{7E0B56FD-7324-4898-BEDB-2F56A843BB0F},2005-07-01
29499,527,Main Office,{C90CB0C3-976A-4075-A2B9-13136F4F1A92},2006-09-01
29502,893,Main Office,{8ACB1C6A-7CDF-417B-AFD5-9F4C642F3C7E},2007-07-01


Product_ID,Name,Product_Number,Color,Standard_Cost,List_Price,Size,Weight,Product_Category_ID,Product_Model_ID,Sell_Start_Date,Sell_End_Date,Discontinued_Date,rowguid,Modified_Date
680,HL Road Frame - Black,58,FR-R92B-58,null,1059.0,31,1431.0,5,58,1016-01-01,null,null,2002-06-01 00:00:00,null
706,HL Road Frame - Red,58,FR-R92R-58,null,1059.0,31,1431.0,5,58,1016-01-01,null,null,2002-06-01 00:00:00,null
707,Sport-100 Helmet,Red,HL-U509-R,null,13.0,0863,34.0,99,null,null,null,2005-07-01,null,null
708,Sport-100 Helmet,Black,HL-U509,null,13.0,0863,34.0,99,null,null,null,2005-07-01,null,null
709,Mountain Bike Socks,M,SO-B909-M,null,3.0,3963,9.0,5,null,null,null,2005-07-01,2006-06-30 00:00:00,null
710,Mountain Bike Socks,L,SO-B909-L,null,3.0,3963,9.0,5,null,null,null,2005-07-01,2006-06-30 00:00:00,null
711,Sport-100 Helmet,Blue,HL-U509-B,null,13.0,0863,34.0,99,null,null,null,2005-07-01,null,null
712,AWC Logo Cap,CA-1098,Multi,6.0,9223.0,8,99.0,null,null,null,2005-07-01,null,null,null
713,Long-Sleeve Logo Jersey,S,LJ-0192-S,null,38.0,4923,49.0,99,null,null,null,2005-07-01,null,null
714,Long-Sleeve Logo Jersey,M,LJ-0192-M,null,38.0,4923,49.0,99,null,null,null,2005-07-01,null,null


Product_Category_ID,Parent_Product_Category_ID,Name,rowguid,Modified_Date
1,null,Bikes,{CFBDA25C-DF71-47A7-B81B-64EE161AA37C},2002-06-01
2,null,Components,{C657828D-D808-4ABA-91A3-AF2CE02300E9},2002-06-01
3,null,Clothing,{10A7C342-CA82-48D4-8A38-46A2EB089B74},2002-06-01
4,null,Accessories,{2BE3BE36-D9A2-4EEE-B593-ED895D97C2A6},2002-06-01
5,1,Mountain Bikes,{2D364ADE-264A-433C-B092-4FCBF3804E01},2002-06-01
6,1,Road Bikes,{000310C0-BCC8-42C4-B0C3-45AE611AF06B},2002-06-01
7,1,Touring Bikes,{02C5061D-ECDC-4274-B5F1-E91D76BC3F37},2002-06-01
8,2,Handlebars,{3EF2C725-7135-4C85-9AE6-AE9A3BDD9283},2002-06-01
9,2,Bottom Brackets,{A9E54089-8A1E-4CF5-8646-E3801F685934},2002-06-01
10,2,Brakes,{D43BA4A3-EF0D-426B-90EB-4BE4547DD30C},2002-06-01


Product_Description_ID,Description,rowguid,Modified_Date
3,Chromoly steel.,{301EED3A-1A82-4855-99CB-2AFE8290D641},2007-06-01
4,Aluminum alloy cups; large diameter spindle.,{DFEBA528-DA11-4650-9D86-CAFDA7294EB0},2007-06-01
5,Aluminum alloy cups and a hollow axle.,{F7178DA7-1A7E-4997-8470-06737181305E},2007-06-01
8,Suitable for any type of riding,on or off-road. Fits any budget. Smooth-shifting with a comfortable ride.,null
64,This bike delivers a high-level of performance on a budget. It is responsive and maneuverable,and offers peace-of-mind when you decide to go off-road.,null
88,For true trail addicts. An extremely durable bike that will go anywhere and keep you in control on challenging terrain - without breaking your budget.,{4C1AD253-357E-4A98-B02E-02180AA406F6},2007-06-01
128,Serious back-country riding. Perfect for all levels of competition. Uses the same HL Frame as the Mountain-100.,{130709E6-8512-49B9-9F62-1F5C99152056},2008-03-11
168,Top-of-the-line competition mountain bike. Performance-enhancing options include the innovative HL Frame,super-smooth front suspension,null
170,Suitable for any type of off-road trip. Fits any budget.,{EA772412-6369-4416-9CC9-C1A5D1FF9C52},2007-06-01
209,Entry level adult bike; offers a comfortable ride cross-country or down the block. Quick-release hubs and rims.,{F5FF5FFD-CB7C-4AD6-BBC9-4D250BB6E98D},2007-06-01


Product_Model_ID,Name,rowguid,Modified_Date
1,Classic Vest,{29321D47-1E4C-4AAC-887C-19634328C25E},2007-06-01
2,Cycling Cap,{474FB654-3C96-4CB9-82DF-2152EEFFBDB0},2005-06-01
3,Full-Finger Gloves,{A75483FE-3C47-4AA4-93CF-664B51192987},2006-06-01
4,Half-Finger Gloves,{14B56F2A-D4AA-40A4-B9A2-984F165ED702},2006-06-01
5,HL Mountain Frame,{FDD5407B-C2DB-49D1-A86B-C13A2E3582A2},2005-06-01
6,HL Road Frame,{4D332ECC-48B3-4E04-B7E7-227F3AC2A7EC},2002-05-02
7,HL Touring Frame,{D60ED2A5-C100-4C54-89A1-531404C4A20F},2009-05-16
8,LL Mountain Frame,{65BF3F6D-BCF2-4DB6-8515-FC5C57423037},2006-11-20
9,LL Road Frame,{DDC67A2F-024A-4446-9B54-3C679BABA708},2005-06-01
10,LL Touring Frame,{66C63844-2A24-473C-96D5-D3B3FD57D834},2009-05-16


Product_Model_ID,Product_Description_ID,Culture,rowguid,Modified_Date
1,1199,en,{4D00B649-027A-4F99-A380-F22A46EC8638},2007-06-01
1,1467,ar,{7DE7204E-4EFC-40F7-B9E6-0CEB4162399C},2007-06-01
1,1589,fr,{20BFFCF4-BFA6-400B-BF34-9A689779EAE4},2007-06-01
1,1712,th,{AF18BC29-F378-4512-A59B-D55217F4F82B},2007-06-01
1,1838,he,{23830AC3-3B16-49D1-9BA1-3A8996F0FA7A},2007-06-01
1,1965,zh-cht,{14ED10B6-0ACF-4D67-9760-6CC622C1F78E},2007-06-01
2,1210,en,{623741C2-74BD-4CE1-A554-1FE8C0606B4B},2007-06-01
2,1476,ar,{1451EB3E-BB1D-45BE-8633-32938460AE99},2007-06-01
2,1598,fr,{6D7B1148-6159-4EC5-B469-D106FFEF6CA5},2007-06-01
2,1721,th,{6AFCD17B-E1ED-4D7B-A695-CD9E76B327C9},2007-06-01


Sales_Order_ID,Sales_Order_Detail_ID,Order_Qty,Product_ID,Unit_Price,Unit_Price_Discount,Line_Total,rowguid,Modified_Date
71774,110562,1,836,356.0,898.0,0.0,356,null
71774,110563,1,822,356.0,898.0,0.0,356,null
71776,110567,1,907,63.0,9.0,0.0,63,null
71780,110616,4,905,218.0,454.0,0.0,873,null
71780,110617,2,983,461.0,694.0,0.0,923,null
71780,110618,6,988,112.0,998.0,0.0,4,null
71780,110619,2,748,818.0,7.0,0.0,1637,null
71780,110620,1,990,323.0,994.0,0.0,323,null
71780,110621,1,926,149.0,874.0,0.0,149,null
71780,110622,1,743,809.0,76.0,0.0,809,null


Sales_Order_ID,Revision_Number,Order_Date,Due_Date,Ship_Date,Status,Online_Order_Flag,Sales_Order_Number,Purchase_Order_Number,Account_Number,Customer_ID,Ship_To_Address_ID,Bill_To_Address_ID,Ship_Method,Credit_Card_Approval_Code,Sub_Total,Tax_Amt,Freight,Total_Due,Comment,rowguid,Modified_Date
71774,2,2008-06-01,2008-06-13,2008-06-08,5,false,SO71774,PO348186287,10-4020-000609,29847,1092,1092,CARGO TRANSPORT 5,880,3484.0,70.0,4279.0,22.0,0087,972,null
71776,2,2008-06-01,2008-06-13,2008-06-08,5,false,SO71776,PO19952192051,10-4020-000106,30072,640,640,CARGO TRANSPORT 5,78,81.0,6.0,3048.0,1.0,9703,87,0851-01-01
71780,2,2008-06-01,2008-06-13,2008-06-08,5,false,SO71780,PO19604173239,10-4020-000340,30113,653,653,CARGO TRANSPORT 5,38418,6895.0,3073.0,4952.0,960.0,4672,42452,6519-01-01
71782,2,2008-06-01,2008-06-13,2008-06-08,5,false,SO71782,PO19372114749,10-4020-000582,29485,1086,1086,CARGO TRANSPORT 5,39785,3304.0,3182.0,8264.0,994.0,6333,43962,7901-01-01
71783,2,2008-06-01,2008-06-13,2008-06-08,5,false,SO71783,PO19343113609,10-4020-000024,29957,992,992,CARGO TRANSPORT 5,83858,4261.0,6708.0,6741.0,2096.0,4607,92663,5609-01-01
71784,2,2008-06-01,2008-06-13,2008-06-08,5,false,SO71784,PO19285135919,10-4020-000448,29736,659,659,CARGO TRANSPORT 5,108561,8317.0,8684.0,9465.0,2714.0,0458,119960,null
71796,2,2008-06-01,2008-06-13,2008-06-08,5,false,SO71796,PO17052159664,10-4020-000420,29660,1058,1058,CARGO TRANSPORT 5,57634,6342.0,4610.0,7707.0,1440.0,8659,63686,2708-01-01
71797,2,2008-06-01,2008-06-13,2008-06-08,5,false,SO71797,PO16501134889,10-4020-000142,29796,642,642,CARGO TRANSPORT 5,78029,6898.0,6242.0,3752.0,1950.0,7422,86222,8072-01-01
71815,2,2008-06-01,2008-06-13,2008-06-08,5,false,SO71815,PO13021155785,10-4020-000276,30089,1034,1034,CARGO TRANSPORT 5,1141,5782.0,91.0,3263.0,28.0,5395,1261,null
71816,2,2008-06-01,2008-06-13,2008-06-08,5,false,SO71816,PO12992180445,10-4020-000295,30027,1038,1038,CARGO TRANSPORT 5,3398,1659.0,271.0,8533.0,84.0,9541,3754,9733-01-01
